## Chest X-ray pneumonia detection model

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

import tensorflow as tf
from tensorflow import keras

In [ ]:
data_dir_train ='.\\dataset\\train'
data_dir_test ='.\\dataset\\test'

In [ ]:
import pathlib
data_dir_train=pathlib.Path(data_dir_train)
data_dir_test=pathlib.Path(data_dir_test)

In [ ]:
train_images_dic = {
'NORMAL': list(data_dir_train.glob('NORMAL/*')),
'BACTERIA': list(data_dir_train.glob('PNEUMONIA/BACTERIA*')),
'VIRUS': list(data_dir_train.glob('PNEUMONIA/VIRUS*'))
}

In [ ]:
labels_dic = {
'NORMAL': 0,
'BACTERIA': 1,
'VIRUS': 2,
}

In [ ]:
img = cv2.imread(str(train_images_dic['BACTERIA'][0]))
img.shape

(1032, 1152, 3)

In [ ]:
X_train, y_train = [], []

for label, images in train_images_dic.items():
    print(label, len(images))

    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(180,180))
        X_train.append(resized_img)
        y_train.append(labels_dic[label])  

NORMAL 1349
BACTERIA 2538
VIRUS 1345


5232 training images

In [ ]:
test_images_dic = {
'NORMAL': list(data_dir_test.glob('NORMAL/*')),
'BACTERIA': list(data_dir_test.glob('PNEUMONIA/BACTERIA*')),
'VIRUS': list(data_dir_test.glob('PNEUMONIA/VIRUS*'))
}

In [ ]:
X_test, y_test = [], []

for label, images in test_images_dic.items():
    print(label, len(images))

    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img,(180,180))
        X_test.append(resized_img)
        y_test.append(labels_dic[label])  

NORMAL 234
BACTERIA 242
VIRUS 148


624 test images

In [ ]:
len(X_train), len(X_test)

(5232, 624)

In [ ]:
X_train = np.array(X_train) 
X_test = np.array(X_test) 
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
X_train_scaled = X_train / 255 
X_test_scaled = X_test / 255 

X_train_scaled

array([[[[0.36078431, 0.36078431, 0.36078431],
         [0.39215686, 0.39215686, 0.39215686],
         [0.37647059, 0.37647059, 0.37647059],
         ...,
         [0.56470588, 0.56470588, 0.56470588],
         [0.64705882, 0.64705882, 0.64705882],
         [0.58039216, 0.58039216, 0.58039216]],

        [[0.37254902, 0.37254902, 0.37254902],
         [0.35686275, 0.35686275, 0.35686275],
         [0.38823529, 0.38823529, 0.38823529],
         ...,
         [0.60392157, 0.60392157, 0.60392157],
         [0.64313725, 0.64313725, 0.64313725],
         [0.3254902 , 0.3254902 , 0.3254902 ]],

        [[0.37647059, 0.37647059, 0.37647059],
         [0.36470588, 0.36470588, 0.36470588],
         [0.37647059, 0.37647059, 0.37647059],
         ...,
         [0.65882353, 0.65882353, 0.65882353],
         [0.38431373, 0.38431373, 0.38431373],
         [0.34901961, 0.34901961, 0.34901961]],

        ...,

        [[0.        , 0.        , 0.        ],
         [0.        , 0.        , 0.        ]

In [ ]:
model = keras.models.Sequential([

    keras.layers.Conv2D(30, (3,3), padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    keras.layers.Dropout(0.5),
    keras.layers.Conv2D(50, (3,3), padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2,2)),

    # Dense network
    keras.layers.Flatten(),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dense(3, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train_scaled, y_train, epochs=5, validation_split=0.2)

Epoch 1/5
131/131 [==============================] - 220s 2s/step - loss: 0.7812 - accuracy: 0.7756 - val_loss: 1.6093 - val_accuracy: 0.0000e+00
Epoch 2/5
131/131 [==============================] - 212s 2s/step - loss: 0.3486 - accuracy: 0.8858 - val_loss: 1.8801 - val_accuracy: 0.0029
Epoch 3/5
131/131 [==============================] - 209s 2s/step - loss: 0.3029 - accuracy: 0.9001 - val_loss: 1.7553 - val_accuracy: 0.1165
Epoch 4/5
131/131 [==============================] - 209s 2s/step - loss: 0.2848 - accuracy: 0.9006 - val_loss: 1.9256 - val_accuracy: 0.1356
Epoch 5/5
131/131 [==============================] - 203s 2s/step - loss: 0.2516 - accuracy: 0.9109 - val_loss: 1.9913 - val_accuracy: 0.1366


In [ ]:
model.evaluate(X_test_scaled, y_test)

20/20 [==============================] - 7s 333ms/step - loss: 0.8751 - accuracy: 0.6218


[0.8750855922698975, 0.6217948794364929]

In [ ]:
y_pred = []

for ele in model.predict(X_test):
       y_pred.append(np.argmax(ele))

y_pred[:5]       

20/20 [==============================] - 7s 341ms/step


[0, 1, 0, 0, 1]

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.96      0.54      0.69       234
           1       0.50      0.98      0.66       242
           2       0.00      0.00      0.00       148

    accuracy                           0.58       624
   macro avg       0.49      0.51      0.45       624
weighted avg       0.55      0.58      0.52       624

